In [1]:
import pandas as pd 

df=pd.read_csv("C://Users//HP//Desktop//data.csv",encoding= 'unicode_escape')

df



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
import plotly.express as px

# just month, time doesn't matter
df["Date"] = pd.to_datetime(df["InvoiceDate"]).dt.date - pd.offsets.MonthBegin(1)
# work out when customer was first a customer to define which cohort
df2 = df.merge(
    df.groupby(["CustomerID"], as_index=False).agg(Cohort=("Date", "min")),
    on="CustomerID",
)
# months between cohort start and invoice date
df2["Month"] = df2["Date"].dt.to_period("M").view(dtype="int64") - df2[
    "Cohort"
].dt.to_period("M").view(dtype="int64")

df_cohort = (
    df2.groupby(["Cohort", "Month"])
    .apply(lambda d: (d["Quantity"] * d["UnitPrice"]).sum())
    .unstack("Month")
)

# rebase as percentage as per referenced example
for c in df_cohort.columns[1:]:
    df_cohort[c] = df_cohort[c] / df_cohort[0]
df_cohort[0] = 1

# now the easy bit - generate a figure
px.imshow(
    df_cohort, text_auto=".2%", color_continuous_scale="greens", range_color=[0, 1]
).update_xaxes(side="top", dtick=1).update_yaxes(dtick="M1")
